In [1]:
#!pwsh
## Set your own path !!
cd  C:\Users\mcp\Documents\GitHub\PowerShellNotebook
import-module .\PowerShellNotebook.psd1 -force


# ConvertFrom-NotebookToMarkdown 

In [1]:
#!pwsh
ConvertFrom-NotebookToMarkdown -Path .\__tests__\DotNetInteractiveNotebooks\TestWithInvokePS.ipynb -destination demo.md -Includeoutput

    Directory: C:\Users\mcp\Documents\GitHub\PowerShellNotebook


Mode                 LastWriteTime         Length Name


----                 -------------         ------ ----


-a---          02/10/2020    08:15            595 demo.md


## Changes
1. Now Supports -Path or -Fullname as a the name of the source file
2. Now Supports -Destination  (if not specified uses .\\\<name\>.ipynb)
3. Now supports [switch]-IncludeOutput - Puts output in a "preformated" block - see also get-NotebookContent 

result will be 
```powershell
$PSVersionTable
```

```
Name                           Value
----                           -----
PSVersion                      7.0.0
PSEdition                      Core
GitCommitId                    7.0.0
OS                             Microsoft Windows 10.0.19041
Platform                       Win32NT
PSCompatibleVersions           {1.0, 2.0, 3.0, 4.0…}
PSRemotingProtocolVersion      2.3
SerializationVersion           1.1.0.1
WSManStackVersion              3.0
```

```powershell
1..10 | % {
    $_ * 2
}
```

```
2
4
6
8
10
12
14
16
18
2
```

```powershell

```



# Export-NotebookToPowerShellScript 

In [1]:
#!pwsh
dir .\__tests__\DotNetInteractiveNotebooks\*.ipynb | Export-NotebookToPowerShellScript -AsText -IncludeTextCells

<#
    Created from: C:\Users\mcp\Documents\GitHub\PowerShellNotebook\__tests__\DotNetInteractiveNotebooks\TestWithInvokePS.ipynb
    Created by:   Export-NotebookToPowerShellScript
    Created on:   02 October 2020 08:06
#>



$PSVersionTable


<# #>


1..10 | % {
    $_ * 2
}


## Changes 
1.  Header changed to 
    * Align text
    * Close gap between created by and created on
    * Format date as _local long date_ so month is (long has month as words so unambiguous Previously date was culture invariant i.e. US-style 10/2/2020 which reads 10th Feb in the rest of the world. 
2. Now Supports piped input
3. Now Supports -AsText swtich
4. (Breaking) IncludeTextCells is now a switch, previously was boolean
5. Now if two consequtive cells are code, outputs "<# #>" between them, and does not output an empty final cell (this would cause a <# #> at the end of files)


# Get-NotebookContent

In [1]:
#!pwsh
Get-NotebookContent .\__tests__\DotNetInteractiveNotebooks\TestWithInvokePS.ipynb -Includeoutput -NoMarkdown 

NoteBookName           Type Source                   Output


------------           ---- ------                   ------


TestWithInvokePS.ipynb code $PSVersionTable          …


TestWithInvokePS.ipynb code 1..10 | % {…             2…


TestWithInvokePS.ipynb code                          


## Changes 
1. Now has Aliases for JustCode and JustMarkdown and these two are mutually exclusive
2. Now has -Include output switch. (can see the first Item of the second cell). Include output tries to merge multiple text output rows which .NET Interactive generates into a single block. It's been OK with what I've tested but it may need more work to make it properly robust

**To do** Piping files in doesn't look like it will work.

# New-psNotebook    

In [1]:
#!pwsh
$pssession =  New-PSSession -ComputerName localhost  -EnableNetworkAccess

New-PSNotebook {
    Add-NotebookCode "#!About" 
    Add-NotebookCode -Verbose '#!time
foreach ($i in 1..3) {$i; Sleep 1}' 
}  -NoteBookName demo.ipynb -DotNetInteractive -IncludeCodeResults -RunSpace $pssession.Runspace

Remove-PSSession $pssession

#So I can paste the result 
ConvertFrom-NotebookToMarkdown .\demo.ipynb -Includeoutput -AsText | clip

VERBOSE: 1
2
3

Wall time 3,086ms


## Changes 
1. If `New-PSNotebook` is told to include code results `Add-NotebookCode` sends code results to verbose.
2. If not including code results `Add-NotebookCode` can be given a complex object not just a text stream to include in output. This needs more testing but is designed to support HTML blocks and similar.  
3. Now supports .NET-Interactve format notebooks (moved the templates to the top of PowerShellNotebookDSL.ps1).
4. Now supports being passed a runspace - remoting FTW !
5. Now takes the absence of a notebookname as impying `-AsText`(in the code block above I could omit the notebook name go straight to | clip without saving a file ). Previously it loooked like specifying neither would cause the file to be named just ".ipynb" 
6. Understands *magic commands* : #!time, #!about (both processed) and #!pwsh (removed).

Coded above put built this notebook:
```powershell
#!About
```

```
PowerShell 5.1.19041.1 on FLATFISH, Microsoft Windows NT 10.0.19041.0
```

```powershell
#!time
foreach ($i in 1..3) {$i; Sleep 1}
```

```
1
2
3

Wall time 3,116ms
```

